In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import pandas as pd

# creating Graphs using NetworkX:

In [ ]:
# Undirected Graph:
G = nx.Graph()
# create nodes:
List_of_nodes = [1, 2, 3, 5, "Jonas", "Sven"]
G.add_nodes_from(List_of_nodes)
nx.draw_networkx(G)
plt.show()

In [ ]:
# create edges:
List_of_edges = [(1,2), (2,"Jonas"), (1,3), (3,5), (2,3), ("Sven",1), ("Sven",2), ("Sven",3), ("Sven",5)]
G.add_edges_from(List_of_edges)
nx.draw_networkx(G)
plt.show()

In [ ]:
# remove nodes:
G.remove_node(2)
nx.draw_networkx(G)
plt.show()

In [ ]:
# create random graph:
H = nx.Graph()
# Graph can be created by only adding edges:
for i in range(0, 10):
    H.add_edge(random.randrange(0,10,1), random.randrange(0,10,1))
nx.draw_networkx(H)
plt.show()
H.nodes()

In [ ]:
# Directed Graph:
I = nx.DiGraph()
List_of_edges = [("Max","Peter"), ("Peter","Jonas"), ("Max","Till"), ("Till","Luisa"), ("Jonas","Luisa"), 
                 ("Peter","Till"), ("Sara","Max"), ("Sara","Peter"), ("Sara","Till"), ("Sara","Luisa")]
I.add_edges_from(List_of_edges)
nx.draw_networkx(I)
plt.show()
plt.title("distribution of exercise solutions through students")

In [ ]:
# Multigraph:
J = nx.MultiDiGraph()
List_of_edges = [("Max","Peter"), ("Peter","Jonas"), ("Max","Till"), ("Till","Luisa"), ("Jonas","Luisa"), 
                 ("Peter","Till"), ("Sara","Max"), ("Sara","Peter"), ("Sara","Till"), ("Sara","Luisa")]
J.add_edges_from(List_of_edges)
J.add_edge("Sara","Peter")
nx.draw_networkx(J)
plt.show()
plt.title("distribution of exercise solutions through students")
J.edges()

# analyzing Networks using NetworkX:

In [ ]:
I = I.to_undirected()
nx.draw_networkx(I)
plt.show()
plt.title("example social network")
# analyze graph I
print("number of nodes in I: %d" % I.number_of_nodes())
print("number of edges in I: %d" % I.number_of_edges())
print("Sara is friends with:") 
print(I.neighbors("Sara"))
print("degree of nodes in I:")
I.degree()

In [ ]:
# analyze graph H
print(nx.is_connected(H))
print(nx.is_directed(H))

# Network algorithm on example data:

In [ ]:
b = nx.betweenness_centrality(I, k = None, normalized = True, endpoints = False, seed = None)
#d = nx.degree_centrality(I)
#c = nx.closeness_centrality(I)
d = {"Betweenness": b}#, "Degree centrality": d, "Closeness centrality": c}
pd.DataFrame(d)

In [ ]:
def betweenness_centrality(G, k=None, normalized=True, weight=None, endpoints=False, seed=None):
    """
    Parameters
    ----------
    G : graph
      A NetworkX graph

    k : int, optional (default=None)
      If k is not None use k node samples to estimate betweenness.
      The value of k <= n where n is the number of nodes in the graph.
      Higher values give better approximation.

    normalized : bool, optional
      If True the betweenness values are normalized by `2/((n-1)(n-2))`
      for graphs, and `1/((n-1)(n-2))` for directed graphs where `n`
      is the number of nodes in G.

    weight : None or string, optional
      If None, all edge weights are considered equal.
      Otherwise holds the name of the edge attribute used as weight.

    endpoints : bool, optional
      If True include the endpoints in the shortest path counts.

    Returns
    -------
    nodes : dictionary
       Dictionary of nodes with betweenness centrality as the value.

    Notes
    -----
    The algorithm is from Ulrik Brandes [1]_.
    See [4]_ for the original first published version and [2]_ for details on
    algorithms for variations and related metrics.

    For approximate betweenness calculations set k=#samples to use
    k nodes ("pivots") to estimate the betweenness values. For an estimate
    of the number of pivots needed see [3]_.

    For weighted graphs the edge weights must be greater than zero.
    Zero edge weights can produce an infinite number of equal length
    paths between pairs of nodes.

    References
    ----------
    .. [1] Ulrik Brandes:
       A Faster Algorithm for Betweenness Centrality.
       Journal of Mathematical Sociology 25(2):163-177, 2001.
       http://www.inf.uni-konstanz.de/algo/publications/b-fabc-01.pdf
    .. [2] Ulrik Brandes:
       On Variants of Shortest-Path Betweenness
       Centrality and their Generic Computation.
       Social Networks 30(2):136-145, 2008.
       http://www.inf.uni-konstanz.de/algo/publications/b-vspbc-08.pdf
    .. [3] Ulrik Brandes and Christian Pich:
       Centrality Estimation in Large Networks.
       International Journal of Bifurcation and Chaos 17(7):2303-2318, 2007.
       http://www.inf.uni-konstanz.de/algo/publications/bp-celn-06.pdf
    .. [4] Linton C. Freeman:
       A set of measures of centrality based on betweenness.
       Sociometry 40: 35–41, 1977
       http://moreno.ss.uci.edu/23.pdf
    """

    betweenness = dict.fromkeys(G, 0.0)  # b[v]=0 for v in G
    if k is None:
        nodes = G
    else:
        random.seed(seed)
        nodes = random.sample(G.nodes(), k)
    for s in nodes:
        # single source shortest paths
        if weight is None:  # use BFS
            S, P, sigma = _single_source_shortest_path_basic(G, s)
        else:  # use Dijkstra's algorithm
            S, P, sigma = _single_source_dijkstra_path_basic(G, s, weight)
        # accumulation
        if endpoints:
            betweenness = _accumulate_endpoints(betweenness, S, P, sigma, s)
        else:
            betweenness = _accumulate_basic(betweenness, S, P, sigma, s)
    # rescaling
    betweenness = _rescale(betweenness, len(G),
                           normalized=normalized,
                           directed=G.is_directed(),
                           k=k)
    return betweenness

# helpers for betweenness centrality

def _single_source_shortest_path_basic(G, s):
    S = []
    P = {}
    for v in G:
        P[v] = []
    sigma = dict.fromkeys(G, 0.0)    # sigma[v]=0 for v in G
    D = {}
    sigma[s] = 1.0
    D[s] = 0
    Q = [s]
    while Q:   # use BFS to find shortest paths
        v = Q.pop(0)
        S.append(v)
        Dv = D[v]
        sigmav = sigma[v]
        for w in G[v]:
            if w not in D:
                Q.append(w)
                D[w] = Dv + 1
            if D[w] == Dv + 1:   # this is a shortest path, count paths
                sigma[w] += sigmav
                P[w].append(v)  # predecessors
    return S, P, sigma


def _single_source_dijkstra_path_basic(G, s, weight='weight'):
    # modified from Eppstein
    S = []
    P = {}
    for v in G:
        P[v] = []
    sigma = dict.fromkeys(G, 0.0)    # sigma[v]=0 for v in G
    D = {}
    sigma[s] = 1.0
    push = heappush
    pop = heappop
    seen = {s: 0}
    c = count()
    Q = []   # use Q as heap with (distance,node id) tuples
    push(Q, (0, next(c), s, s))
    while Q:
        (dist, _, pred, v) = pop(Q)
        if v in D:
            continue  # already searched this node.
        sigma[v] += sigma[pred]  # count paths
        S.append(v)
        D[v] = dist
        for w, edgedata in G[v].items():
            vw_dist = dist + edgedata.get(weight, 1)
            if w not in D and (w not in seen or vw_dist < seen[w]):
                seen[w] = vw_dist
                push(Q, (vw_dist, next(c), v, w))
                sigma[w] = 0.0
                P[w] = [v]
            elif vw_dist == seen[w]:  # handle equal paths
                sigma[w] += sigma[v]
                P[w].append(v)
    return S, P, sigma


def _accumulate_basic(betweenness, S, P, sigma, s):
    delta = dict.fromkeys(S, 0)
    while S:
        w = S.pop()
        coeff = (1.0 + delta[w]) / sigma[w]
        for v in P[w]:
            delta[v] += sigma[v] * coeff
        if w != s:
            betweenness[w] += delta[w]
    return betweenness


def _accumulate_endpoints(betweenness, S, P, sigma, s):
    betweenness[s] += len(S) - 1
    delta = dict.fromkeys(S, 0)
    while S:
        w = S.pop()
        coeff = (1.0 + delta[w]) / sigma[w]
        for v in P[w]:
            delta[v] += sigma[v] * coeff
        if w != s:
            betweenness[w] += delta[w] + 1
    return betweenness


def _rescale(betweenness, n, normalized, directed=False, k=None):
    if normalized is True:
        if n <= 2:
            scale = None  # no normalization b=0 for all nodes
        else:
            scale = 1.0 / ((n - 1) * (n - 2))
    else:  # rescale by 2 for undirected graphs
        if not directed:
            scale = 1.0 / 2.0
        else:
            scale = None
    if scale is not None:
        if k is not None:
            scale = scale * n / k
        for v in betweenness:
            betweenness[v] *= scale
    return betweenness

In [ ]:
"""
Centrality measures of Krackhardt social network.
"""
# Author: Aric Hagberg (hagberg@lanl.gov)
# Date: 2005-05-12 14:33:11 -0600 (Thu, 12 May 2005)
# Revision: 998

#    Copyright (C) 2004-2016 by
#    Aric Hagberg <hagberg@lanl.gov>
#    Dan Schult <dschult@colgate.edu>
#    Pieter Swart <swart@lanl.gov>
#    All rights reserved.
#    BSD license.

J = krackhardt_kite_graph()
nx.draw_networkx(J, with_labels = True)
plt.title('Krackhardt Graph')
plt.show()
b = nx.betweenness_centrality(J)
d = nx.degree_centrality(J)
c = nx.closeness_centrality(J)
d = {"Betweenness": b, "Degree centrality": d, "Closeness centrality": c}
pd.DataFrame(d)